# Projet de RIW

Par Antoine Apollis, Marine Sobas et Paul Viossat

## Installation

In [6]:
!pip install --user nltk

In [1]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/marine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Expressions régulières et constantes

In [2]:
from re import compile

punctuation_regex = compile(r'[,.;:?!—–&]?[" ]+|["\']')
number_regex = compile("^[0-9,.]*$")
index_regex = compile(r'(\S+), \d+ \| (\(\S+, \d+ ;.*)')
doc_occ_pos_regex = compile(r'\((\S+), (\d+) ; ((?:\d+ ?)+)\) ')

In [3]:
INDEX_FILENAME = 'INDEX'

In [4]:
QUERIES = {2: 'stanford class', 3: 'stanford students', 4: 'very cool', 8: 'stanford computer science'}

In [5]:
RESULTS_LENGTH = {2: 6094, 3: 22335, 4: 63, 8: 4232}

## Fonctions de traitement du texte

In [6]:
from nltk.tokenize import word_tokenize

# Tokenizes a character string
def tokenize(s):
    return [w.lower() for w in punctuation_regex.split(s) if len(w) > 1]

In [7]:
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

# Removes stop words (from NLTK) from a list of tokens
def remove_stop_words(tokens):
    return [w for w in tokens if not w in stopwords]

In [8]:
def remove_numbers(array):
    [w for w in filtered_sentence if not number_regex.match(w)]

In [9]:
from nltk.stem import WordNetLemmatizer, PorterStemmer

lemmatizer = WordNetLemmatizer()

# Lemmatizes a list of tokens
def lemmatize(tokens):
    return [lemmatizer.lemmatize(w) for w in tokens]

stemmer = PorterStemmer()

# Stems a list of tokens
def stem(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in tokens]

## Construction de l’index

In [10]:
# Saves an index in a file
def save_index(index):
    with open(INDEX_FILENAME, 'w') as f:
        for word in index:
            f.write(f'{word}, {len(index[word])} | ')
            for (document, tokens) in index[word].items():
                f.write(f'({document}, {tokens[0]} ; {" ".join(map(str, tokens[1]))}) ')
            f.write('\n')

In [11]:
# Loads an index from a file
def load_index():
    with open(INDEX_FILENAME) as f:
        inverted_index = dict()
        for l in f:
            m = index_regex.match(l)
            inverted_index[m.group(1)] = dict(map(lambda t: (t[0], [int(t[1]), list(map(int, t[2].split(' ')))]), doc_occ_pos_regex.findall(m.group(2))))
    return inverted_index

In [12]:
# Extracts the vocabulary from a (tokenized) colelction
def extract_vocabulary(collection):
    vocabulary = set()
    for tokens in collection.values():
        for t in tokens:
            vocabulary.add(t)
    return vocabulary

In [13]:
# Loads a document from the disk
def load_document(filename):
    with open(filename) as f:
        return f.read().rstrip()

In [14]:
from time import time
from os import listdir
COLLECTION_FILENAME = "collection"

# Loads (and tokenizes) a collection from a directory
def load_collection(directory,COLLECTION_FILENAME,forcebuild=False):
    if not forcebuild and isfile(COLLECTION_FILENAME):
        filehandler = open(COLLECTION_FILENAME, 'rb')
        collection=  pickle.load(filehandler)
    else:
        print('Chargement de la collection : ', end='')
        collection = dict()
        for sub_dir in listdir(directory):
            if sub_dir != ".DS_Store":
                print(sub_dir)
                path = directory + '/' + sub_dir
                for filename in listdir(path):
                    fullpath = './' + path + '/' + filename
                    collection[fullpath] = list()
        print('fait')
        ndocuments = len(collection)
        print(f'La collection comporte {ndocuments} documents.\n======')
        progress = 0
        step = ndocuments // 10
        nextstep = step
        chrono = time()
        for fullpath in collection.keys():
            collection[fullpath] = stem(remove_stop_words(tokenize(load_document(fullpath))))
            progress += 1
            if progress > nextstep:
                print(f'Traitement de la collection en cours : encore {round((time() - chrono) / nextstep * (ndocuments - progress) / 60)} min')
                nextstep += step
        filehander = open('collection', 'wb') 
        pickle.dump(collection, filehander)
    return collection

In [15]:
# Builds an inverted index from a collection and a vocabulary
def build_index(collection, vocabulary):
    index = {word: dict() for word in vocabulary}
    progress = 0
    ndocuments = len(collection)
    step = ndocuments // 10
    nextstep = step
    chrono = time()
    for (document, tokens) in collection.items():
        i = 0
        for t in tokens:
            if document in index[t]:
                index[t][document][0] += 1
                index[t][document][1].append(i)
            else:
                index[t][document] = [1, [i]]
            i += 1
        progress += 1
        if progress > nextstep:
            print(f'Création de l’index en cours : encore {round((time() - chrono) / nextstep * (ndocuments - progress))} s')
            nextstep += step
    save_index(index)
    print('\nIndex créé et enregistré')
    return index

In [16]:
from time import time
from os.path import isfile, getsize

# Builds an inverted index for a given directory
def build_inverted_index(directory, forcebuild=False):
    if not forcebuild and isfile(INDEX_FILENAME):
        return load_index()
    fullchrono = time()
    collection = load_collection(directory)
    print('======\nCréation du vocabulaire : ', end='')
    vocabulary = extract_vocabulary(collection)
    print('fait')
    print(f'Le vocabulaire comporte {len(vocabulary)} éléments.\n======')
    index = build_index(collection, vocabulary)
    print(f'======\nL’opération complète a nécessité {(time() - fullchrono) / 60:.1f} minutes.')
    print(f'L’index occupe {getsize(INDEX_FILENAME) // 1000} ko.')
    return index

In [17]:
index = build_inverted_index('pa1-data')

## Recherche booléenne

In [18]:
# Performs a boolean AND search query
def boolean_search(query):
    # We stem all the terms of the query.
    terms = list(map(lambda t: stem([t])[0], query.split(' ')))
    if len(terms) < 1:
        return set()
    results = set(index[terms[0]].keys())
    for t in terms[1:]:
        results &= set(index[t].keys())
    return sorted(list(results))

In [19]:
for i, q in QUERIES.items():
    r = len(boolean_search(q))
    if r < RESULTS_LENGTH[i]:
        print(f'Query {i}: missed {100 * (RESULTS_LENGTH[i] - r) / RESULTS_LENGTH[i]:.2f}% of the results')
    elif r > RESULTS_LENGTH[i]:
        print(f'Query {i}: retrieved {100 * (r - RESULTS_LENGTH[i]) / RESULTS_LENGTH[i]:.2f}% of results in excess')
    else:
        print(f'Query {i}: retrieved the correct number of results')

Query 2: retrieved 35.59% of results in excess
Query 3: retrieved 25.12% of results in excess
Query 4: missed 100.00% of the results
Query 8: retrieved 74.93% of results in excess


## Propriétés de la collection

In [20]:
import pickle
collection = load_collection('pa1-data',COLLECTION_FILENAME)

In [21]:
v = sum(map(len, collection.values()))
print(v)

18244627


In [22]:
import numpy as np
from statistics import stdev
v = stdev(map(len, collection.values()))
print(v)

127.06206179096883


La collection comporte donc 98 998 documents, et 18 244 627 mots (après retrait des mots vides).

In [23]:
m = len(index.keys())
print(m)

296631


Le vocabulaire comporte 162 076 mots.

## Modèle vectoriel 

In [24]:
from vectorial_model import processing_vectorial_query,get_stats_collection
weighting_schemes = {"frequency":"FRQ","tf_idf_normalize":"TIN","tf_idf_logarithmic":"TIL","tf_idf_logarithmic_normalize":"TILN"}
q = list(QUERIES.values())[-1]
stats_collection = get_stats_collection(collection)
weighting_scheme_query = weighting_schemes["frequency"]
weighting_scheme_document = weighting_schemes["tf_idf_logarithmic_normalize"]
processed = processing_vectorial_query(q, index, stats_collection, weighting_scheme_document,weighting_scheme_query)

  4%|▍         | 4213/98998 [00:00<00:02, 42124.92it/s]

Computing statistics on the entire collection


100%|██████████| 98998/98998 [00:02<00:00, 47379.14it/s]


## Evaluation des différents modèles

In [26]:
relevant_doc_ids = {}

relevant_doc_ids["boolean"] = {}
for model in weighting_schemes.keys():
    relevant_doc_ids[model] = {}
    for i,query in QUERIES.items() :
        relevant_doc_ids[model][str(i)] = list(processing_vectorial_query(query, index, stats_collection, weighting_scheme_document,weighting_scheme_query).keys())
        relevant_doc_ids["boolean"][str(i)] = boolean_search(query)

In [27]:
from utils import get_queries_output
from eval import mAp

true_results= get_queries_output(directory='Queries/dev_output')
maps = {}
models =  list(weighting_schemes.keys())
models.append("boolean")
for model in models:
    print("Computing the mean average precision for {}".format(model))
    obtained_results = relevant_doc_ids[model]
    maps[model] = mAp(QUERIES,obtained_results,true_results, 10,model,forcebuild= True)

  2%|▏         | 1152/73196 [00:00<00:06, 11518.86it/s]

Computing the mean average precision for frequency
Computing the mean average precision for query stanford class


  2%|▏         | 1142/73002 [00:00<00:06, 11414.62it/s]

Computing the mean average precision for query stanford students


 36%|███▌      | 26136/73002 [02:23<10:01, 77.88it/s]  

KeyboardInterrupt: 